In [1]:
#se importan los paquetes necesarios
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from ipywidgets import interact
from sympy.solvers import solve, linsolve
from sympy import Symbol
from sympy import symarray
from itertools import combinations

In [2]:
#se crea la función que calculará r efectiva
def f_reff(i,x,params):
    r,b,c = params
    
    N=len(r)
        
    r_eff = r[i] 
    for j in range(N):
        r_eff = r_eff + b[i][j]*x[j]
        
    return r_eff

In [3]:
#encontrar las soluciones estacionarias de las ecuaciones de r_eff y las combinaciones de x=1/c
def solut_fact_reff_Nsp2(params):
    r,b,c = params  #se introducen los parámetros en el modelo
    N=len(r)
    
    if any(c[i]==0 for i in range(N)):  #se comprueba que ningun valor de c sea 0
        print('c=0')
        return

    x=symarray("x", N)
    #se generan listas donde posteriormente se almacenarán diversos datos de interés
    r_ef=[]
    x_n=[]
    dx_n=[]
    sols=[]
    Sols=[]
    Sols_dict={}
    ind_cc_dict={}  # diccionario que contiene 0/1 para cada combinación, siendo 1: variable=1/c
    
    for i in range(N):
        r_eff = r[i] 
        for j in range(N):
            r_eff = r_eff + b[i][j]*x[j]  #se calcula r efectiva
        dx = r_eff * x[i]*(1-c[i]*x[i])   #se crea la ecuación con esa r efectiva
        r_ef.append(r_eff)  #se almacena la r efectiva
        dx_n.append(dx)   #se almacena toda la ecuación para esa especie
        x_n.append(x[i])  #se guarda la especie que ese esta estudiando, por ejemplo x_1
    
    sol_cc = [1./c[i] for i in range(N)] #se calculan las carrying capacities 
        
    for nfix in range(0,N): #Se evaluarán las combinaciones posibles cogiendo desde 1 especie hasta N-1 especies, que será el total         
        comb_fix = np.math.factorial(N)/(np.math.factorial(nfix)*np.math.factorial(N-nfix))
        combinat = list(combinations(range(N),nfix)) #se generan las combinaciones de 0 y 1 para nfix, siendo 1 las especies que están en carrying capacity
        Sols=[]
        ind_cc=[]
        for list_fix in combinat:           
            i_cc=[]        
            Eqs=[]
            Xs=[]        
            i_cc = [ 1 if i in list_fix else 0 for i in range(N)]
            for i in range(N):
                if i not in list_fix: #se calculan las ecuaciones de las variables que no están en carrying capacity
                    eq = r_ef[i]
                    for k in list_fix:
                        eq = eq.subs({x[k]:sol_cc[k]})
                    Eqs.append(eq) #se almacena la ecuación generada
                    Xs.append(x[i])                  
                        
            sol_lin = next(iter(linsolve(Eqs,Xs))) #se resuelven las ecuaciones
            sol=[]
            for s in sol_lin:
                sol.append(s)  #se almacenan las soluciones
        
            if sol: #se generará la solución completa, mezclando las soluciones calculadas y las que estan en carrying capacity
                j = 0
                i_cc = [0 for i in range(N)]
                X_fin=[]
                for i in range(N):                    
                    if i in list_fix:
                        X_fin.append(sol_cc[i])
                        i_cc[i]=1  #está en carrying capacity
                    else:
                        X_fin.append(np.float(sol[j]))
                        j +=1
                        i_cc[i]=0  #no está en carrying capacity
                        
                Sols.append(X_fin)  #se almacena la solución entera
                ind_cc.append(i_cc) #se almacená el vector con las especies que están en carrying capacity
                                
        Sols_dict[nfix]= Sols                                
        ind_cc_dict[nfix] = ind_cc
        
    Sols_dict[N] = [ [sol_cc[i] for i in range(N)] ]
    ind_cc_dict[N] = [[1 for i in range(N)]]
    
    Sols_uniq_dict={}
    ind_cc_uniq_dict={}
    for nfix in range(0,N+1):
        ss=0
        Sols_uniq=[]
        ind_cc_uniq=[]
        for s in Sols_dict[nfix]:
            if s not in Sols_uniq:    #se eliminan posiibles soluciones repetidas        
                if all([s[i]>0 for i in range(N) if len(s)>1]):  #se comprueba que todos los valores de la solción sean mayores que 0
                    Sols_uniq.append(s)
                    ind_cc_uniq.append(ind_cc_dict[nfix][ss])
            ss +=1
            
        Sols_uniq_dict[nfix]=Sols_uniq   #lista de soluciones con todos sus valores por encima de 0
        ind_cc_uniq_dict[nfix]=ind_cc_uniq   #lista de vectores que indican qué especies estan en la carrying capacity en cada solución
        #print('nfix: ',nfix)
        #print('Sols_unique_dict: ', Sols_uniq_dict[nfix])
        #print('ind_cc_uniq_dict: ', ind_cc_uniq_dict[nfix])
    
    return(Sols_uniq_dict, ind_cc_uniq_dict)

In [4]:
#función que calcula la matriz Jacobiana cuando ninguna especie está en carrying capacity
def Jacobian_fact_X0_Nsp(X,params):
    r,b,c = params
    
    N=len(r)
    JJ = np.ndarray(shape=(N,N), dtype=float)
    
    for i in range(N):
        for j in range(N):
            JJ[i][j] = b[i][j]*X[i]*(1. - c[i]*X[i]) #se calcula cada valor de la matriz
    
    return JJ

In [5]:
#función que calcula la matriz Jacobiana cuando todas las especies están en carrying capacity
def Jacobian_fact_1C_Nsp(X,params):
    r,b,c = params
    
    N=len(r)
    x_c= np.zeros((N,))
    r_eff = np.zeros((N,))

    for i in range(N):
        r_eff[i]=f_reff(i,X,params)
        
    JJ = np.zeros(shape=(N,N), dtype=float) #se genera la matriz con todos los valores como 0
    for i in range(N):
            JJ[i][i] = -r_eff[i]  #se calcula la diagonal de la matriz que será la única que no sea 0
    return JJ

In [6]:
#función que calcula la matriz Jacobiana cuando hay mezcla de especies en carrying capacity y especies que no.
def Jacobian_fact_1C_X_Nsp(X,cc,params):
    r,b,c = params
    
    N=len(r)
    JJ_cc = np.zeros(shape=(N,N), dtype=float)  #se genera una matriz con todos los valores como 0
    for i in range(N): #para cada fila se estudia cada caso
        if cc[i]: #en carrying capacity
            for j in range(N):
                if j == i:
                    JJ_cc[i][j] = -f_reff(j,X,params)  #se calcula el valor para esa posición
                    
        else:  #no en carrying capacity
            for j in range(N):
                JJ_cc[i][j] = b[i][j]*X[i]*(1.-c[i]*X[i]) #se calcula el valor para esa posición

    return JJ_cc

In [7]:
#función que permite calcular los autovectores de cada solución
def lambdas_fact_X_icc_Nsp(X,i_cc,params):
    list_0 = []
    list_1 = []
    for i in range(len(params[0])):
        list_0.append(0) 
        list_1.append(1)
    if i_cc == list_0:  #ninguna especie esta en carrying capacity
        JJ = Jacobian_fact_X0_Nsp(X,params)
    elif i_cc == list_1:    #todas las especies estan en carrying capacity    
        JJ = Jacobian_fact_1C_Nsp(X,params)
    else:   #mezcla de especies en carrying capacity
        JJ = Jacobian_fact_1C_X_Nsp (X,i_cc,params)
  

    lambdas=np.linalg.eigvals(JJ) #se calculan los autovalores
    return lambdas

### Población 1 (Primer Dataset del artículo)

In [8]:
#se calcula la solución estacionaria y sus autovalores para ver si existe solución estable
r=[0.3, 0.25, 0.36, 0.52, 0.59, 0.46, 0.82, 0.44, 0.22]
b=[[ 2.43435886e-01, -1.95732475e-01,  4.90237021e-03,
         2.91866825e-02,  3.07150441e-04, -1.41175550e-02,
        -3.87079546e-04, -6.74236647e-02, -1.71315036e-04],
       [-1.95732475e-01,  5.64028304e-01, -8.42092410e-02,
        -6.22121972e-02, -7.16102756e-04, -1.11119735e-02,
        -3.36312141e-03, -4.65794894e-02, -1.60103704e-01],
       [ 4.90237021e-03, -8.42092410e-02,  3.69714717e-02,
        -1.73893690e-02, -9.50485010e-04,  5.58443229e-04,
        -4.88200033e-05,  2.32359618e-02,  3.69296681e-02],
       [ 2.91866825e-02, -6.22121972e-02, -1.73893690e-02,
         6.93676410e-02,  1.89860271e-03,  1.09355460e-02,
         5.65232836e-04, -2.61723857e-02, -6.17975307e-03],
       [ 3.07150441e-04, -7.16102756e-04, -9.50485010e-04,
         1.89860271e-03,  5.04159022e-04,  6.05459155e-05,
        -9.55755065e-05, -1.17595178e-03,  1.67656970e-04],
       [-1.41175550e-02, -1.11119735e-02,  5.58443229e-04,
         1.09355460e-02,  6.05459155e-05,  1.49270030e-02,
         2.02974391e-04, -7.40260657e-03,  5.94762250e-03],
       [-3.87079546e-04, -3.36312141e-03, -4.88200033e-05,
         5.65232836e-04, -9.55755065e-05,  2.02974391e-04,
         1.48116485e-04,  1.45061388e-03,  1.52765888e-03],
       [-6.74236647e-02, -4.65794894e-02,  2.32359618e-02,
        -2.61723857e-02, -1.17595178e-03, -7.40260657e-03,
         1.45061388e-03,  1.43869444e-01, -1.98019219e-02],
       [-1.71315036e-04, -1.60103704e-01,  3.69296681e-02,
        -6.17975307e-03,  1.67656970e-04,  5.94762250e-03,
         1.52765888e-03, -1.98019219e-02,  1.41684087e-01]]
c=[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
N= len(r)    
params=(r,b,c)
sols_fact_dict, sols_cc_dict = solut_fact_reff_Nsp2(params)  #se calculan las soluciones estacionarias
X_finite=[]
Xss=[]
Lss=[]
for (ns,list_sols),(ni,list_cc) in zip(sols_fact_dict.items(), sols_cc_dict.items()):
    if list_sols:        
        for X,i_cc in zip(list_sols,list_cc): #se barre la lista de soluciones y de vectores de carrying capacity
            X_finite.append(X)
            lambdas = lambdas_fact_X_icc_Nsp(X,i_cc,params) #se calculan los autovalores
            if all(lambdas[i].real<0 for i in range(len(lambdas))): #se mira si todos tienen parte real negativa
                Xss.append([X[i] for i in range(len(X))])  #se almacena la solución estable
                Lss.append([lambdas[i] for i in range(len(lambdas))])    #se almacena qué especies están en carrying capacity en esa solución

if len (Xss) != 0:
    print (Xss)    #se muestra la solución estable
    print (Lss)    #se muestran los autovalores de dicha solución

[[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]]
[[-0.2999999869000049, -0.2499999733999978, -0.3600000025699992, -0.5200000076000002, -0.5900000006, -0.4599999965499998, -0.82000000062, -0.4399999630000002, -0.21999994439999604]]


In [13]:
#f_out = open('Cambios_en_dataset1_parejas','a')
cambios=0    #contador del total de cambios
contador_p=0  #contador de cambios de negativo a positivo que mantienen una solución estable
contador_n=0  #contador de cambios de positivo a negativo que mantienen una solución estable
sols_dif_p=0  #contador de cambios de negativo a positivo que obtienen una solución estable distinta de la solución: todas las especies en CC
sols_dif_n=0  #contador de cambios de positivo a negativo que obtienen una solución estable distinta de la solución: todas las especies en CC
cambio_n_p=0  #contador de cambios de negativo a positivo
Lss_original=[-0.2999999869000049, -0.2499999733999978, -0.3600000025699992, -0.5200000076000002, -0.5900000006, -0.4599999965499998, -0.82000000062, -0.4399999630000002, -0.21999994439999604] #autovalores de los parámetros originales (obtenidos en la celda anterior)
mas_estable_p=0   #contador de cambios de negativo a positivo que obtienen una solución más estable que la original
mas_estable_n=0   #contador de cambios de positivo a negativo que obtienen una solución más estable que la original
mas_una_solucion=0    #contador de cambios que obtienen más de una solución estable
for i in range(len(b)):
    for j in range(i+1,len(b[i])):   #sustituir esta linea por: for j in range(len(b)) si se desean hacer los cambios de singo individuales
        print(i,j)
        r=[0.3, 0.25, 0.36, 0.52, 0.59, 0.46, 0.82, 0.44, 0.22]
        b=[[ 2.43435886e-01, -1.95732475e-01,  4.90237021e-03,
         2.91866825e-02,  3.07150441e-04, -1.41175550e-02,
        -3.87079546e-04, -6.74236647e-02, -1.71315036e-04],
       [-1.95732475e-01,  5.64028304e-01, -8.42092410e-02,
        -6.22121972e-02, -7.16102756e-04, -1.11119735e-02,
        -3.36312141e-03, -4.65794894e-02, -1.60103704e-01],
       [ 4.90237021e-03, -8.42092410e-02,  3.69714717e-02,
        -1.73893690e-02, -9.50485010e-04,  5.58443229e-04,
        -4.88200033e-05,  2.32359618e-02,  3.69296681e-02],
       [ 2.91866825e-02, -6.22121972e-02, -1.73893690e-02,
         6.93676410e-02,  1.89860271e-03,  1.09355460e-02,
         5.65232836e-04, -2.61723857e-02, -6.17975307e-03],
       [ 3.07150441e-04, -7.16102756e-04, -9.50485010e-04,
         1.89860271e-03,  5.04159022e-04,  6.05459155e-05,
        -9.55755065e-05, -1.17595178e-03,  1.67656970e-04],
       [-1.41175550e-02, -1.11119735e-02,  5.58443229e-04,
         1.09355460e-02,  6.05459155e-05,  1.49270030e-02,
         2.02974391e-04, -7.40260657e-03,  5.94762250e-03],
       [-3.87079546e-04, -3.36312141e-03, -4.88200033e-05,
         5.65232836e-04, -9.55755065e-05,  2.02974391e-04,
         1.48116485e-04,  1.45061388e-03,  1.52765888e-03],
       [-6.74236647e-02, -4.65794894e-02,  2.32359618e-02,
        -2.61723857e-02, -1.17595178e-03, -7.40260657e-03,
         1.45061388e-03,  1.43869444e-01, -1.98019219e-02],
       [-1.71315036e-04, -1.60103704e-01,  3.69296681e-02,
        -6.17975307e-03,  1.67656970e-04,  5.94762250e-03,
         1.52765888e-03, -1.98019219e-02,  1.41684087e-01]]
        c=[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
        #f_out.write(str(i)+str(j)+" = "+str(b[i][j])+" -> ")
        
        b[i][j]=0-b[i][j]   #se realizan los cambios de signo en los parámetros, en este caso por parejas
        b[j][i]=0-b[j][i]   #esta línea se eliminaría para hacer los cambios individuales
        #f_out.write(str(b[i][j])+'\n')
        print(-b[i][j],'->',b[j][i])
        N= len(r)    
        params=(r,b,c)
        sols_fact_dict, sols_cc_dict = solut_fact_reff_Nsp2(params)  #cálculo de soluciones estacionarias
        X_finite=[]
        Xss=[]
        Lss=[]
        for (ns,list_sols),(ni,list_cc) in zip(sols_fact_dict.items(), sols_cc_dict.items()):
            if list_sols:        
                for X,i_cc in zip(list_sols,list_cc):  #se barre la lista de soluciones y de vectores de carrying capacity
                    X_finite.append(X)
                    lambdas = lambdas_fact_X_icc_Nsp(X,i_cc,params)  #se calculan los autovalores
                    if all(lambdas[i].real<0 for i in range(len(lambdas))):  #se mira si todos tienen parte real negativa
                        Xss.append([X[i] for i in range(len(X))])   #se almacena la solución estable
                        Lss.append([lambdas[i] for i in range(len(lambdas))])    #se almacena qué especies están en carrying capacity en esa solución
        
        if len(Xss)>1:
            mas_una_solucion=mas_una_solucion+1    #se comprueba si hay más de una solución estable
            
        if b[i][j]>0:    #cambios de negativo a positivo
            cambio_n_p=cambio_n_p+1
            if len(Xss)!=0:   #si existe solución estable
                #f_out.write(str(Xss)+'\n')
                #f_out.write(str(Lss)+'\n')
                print(Xss)
                print(Lss)
                contador_p=contador_p+1
                if min(Lss_original)>min(Lss[0]): #se comprueba si el valor más negativo de los autovalores originales es mayor que el de los nuevos calculados, si es así, la solución nueva es más estable
                    mas_estable_p=mas_estable_p+1
                    #f_out.write('MAS ESTABLE\n')
                    print('MAS ESTABLE')
                if Xss!=[[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]]:  #se comprueba si la solución es distinta de todos en CC
                    sols_dif_p=sols_dif_p+1
                    
        else:      #cambios de positivo a negativo
            if len(Xss)!=0:
                #f_out.write(str(Xss)+'\n')
                #f_out.write(str(Lss)+'\n')
                print(Xss)
                print(Lss)
                contador_n=contador_n+1
                if min(Lss_original)>min(Lss[0]):
                    mas_estable_n=mas_estable_n+1
                    f_out.write('MAS ESTABLE\n')
                if Xss!=[[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]]:
                    sols_dif_n=sols_dif_n+1
        cambios=cambios+1    

print('El número total de cambios ha sido: ',cambios)
print('\n')
print('El número de cambios de negativo a positivo ha sido: ',cambio_n_p)
print('De los cuales ',contador_p,' han mantenido una solución estable, siendo ',sols_dif_p,' distintas de la solución todas las especies en CC')
print('De estos cambios que han mantenido solución, ',mas_estable_p,' han sido más estables que la solución original')
print('\n')
print('El número de cambios de positivo a negativo ha sido: ',cambios-cambio_n_p)
print('De los cuales ',contador_n,' han mantenido una solución estable, siendo ',sols_dif_n,' distintas de la solución todas las especies en CC')
print('De estos cambios que han mantenido solución, ',mas_estable_n,' han sido más estables que la solución original')
print('\n')
print('El número de cambios con más de una solución es: ',mas_una_solucion)

#f_out.write('El número total de cambios ha sido: '+str(cambios)+'\n')        
#f_out.write('El número de cambios de negativo a positivo ha sido: '+str(cambio_n_p)+'\n')
#f_out.write('\tDe los cuales '+str(contador_p)+' han mantenido una solución estable, siendo '+str(sols_dif_p)+' distintas de la solución todas las especies en CC\n')
#f_out.write('\tDe estos cambios que han mantenido solución, '+str(mas_estable_p)+' han sido más estables que la solución original\n')
#f_out.write('El número de cambios de positivo a negativo ha sido: '+str(cambios-cambio_n_p)+'\n')
#f_out.write('\tDe los cuales '+str(contador_n)+' han mantenido una solución estable, siendo '+str(sols_dif_n)+' distintas de la solución todas las especies en CC\n')
#f_out.write('\tDe estos cambios que han mantenido solución, '+str(mas_estable_n)+' han sido más estables que la solución original\n')
#f_out.write('El número de cambios con más de una solución es: '+str(mas_una_solucion)+'\n')
#f_out.close()

0 1
-0.195732475 -> 0.195732475
[[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]]
[[-39.4464949869, -39.396494973399996, -0.3600000025699992, -0.5200000076000002, -0.5900000006, -0.4599999965499998, -0.82000000062, -0.4399999630000002, -0.21999994439999604]]
MAS ESTABLE
0 2
0.00490237021 -> -0.00490237021
0 3
0.0291866825 -> -0.0291866825
0 4
0.000307150441 -> -0.000307150441
[[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]]
[[-0.23856989870000328, -0.2499999733999978, -0.3600000025699992, -0.5200000076000002, -0.5285699124, -0.4599999965499998, -0.82000000062, -0.4399999630000002, -0.21999994439999604]]
0 5
-0.014117555 -> 0.014117555
[[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]]
[[-3.1235109869000057, -0.2499999733999978, -0.3600000025699992, -0.5200000076000002, -0.5900000006, -3.28351099655, -0.82000000062, -0.4399999630000002, -0.21999994439999604]]
MAS ESTABLE
0 6
-0.000387079546 -> 0.000387079546
[[100.0, 100.0, 100.0, 100.0, 1

[[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.60367987267337]]
[[-0.08604817005210211, -0.2998965674608854, -0.15334858975474575, -0.382293699906477, -0.5162694150535497, -0.5901012117383024, -0.46359045654350906, -0.5135460077018332, -0.4280459413087201]]
7 8
-0.0198019219 -> 0.0198019219
[[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]]
[[-0.2999999869000049, -0.2499999733999978, -0.3600000025699992, -0.5200000076000002, -0.5900000006, -0.4599999965499998, -0.82000000062, -4.400384343000001, -4.180384324399997]]
MAS ESTABLE
El número total de cambios ha sido:  36


El número de cambios de negativo a positivo ha sido:  21
De los cuales  21  han mantenido una solución estable, siendo  0  distintas de la solución todas las especies en CC
De estos cambios que han mantenido solución,  18  han sido más estables que la solución original


El número de cambios de positivo a negativo ha sido:  15
De los cuales  9  han mantenido una solución estable, siendo  1  

-0.00189860271 -> 0.00189860271
